# DVPT 4

In [1]:
import random as rd
import numpy as np
import matplotlib.pyplot as plt
from Bio.Seq import *
from Bio.Alphabet.IUPAC import IUPACAmbiguousDNA, IUPACUnambiguousDNA
import reprlib
import time
from sys import getsizeof

In [59]:
class Genome:
    def __init__(self, sequence):
        self.sequence = sequence
        
    def sequencing(self, read_length=100, reads_nb=5000):
        reads=[]
        for _ in range(reads_nb):
            start = rd.randint(0, len(self.sequence)-1)
            read_seq = self.sequence[start:min(start+100, len(self.sequence))] + self.sequence[0:max(start+100-len(self.sequence), 0)]
            read = Read(read_seq)
            reads.append(read)
        return reads


    
class Read:
    reads = []
    all_kmers = []
    all_km1mers = []
    
    def __init__(self, sequence):
        self.sequence = sequence
        
    def generate_kmers(self, kmers_length):
        """Returns a list of the k-mers (as strings only) included in the read"""
        kmers=[]
        for i in range(len(self.sequence) - kmers_length):
            kmer = self.sequence[i:i+kmers_length]
            kmers.append(kmer)
        return kmers
    
    def generate_all_kmers(cls, kmers_length):
        all_kmers = set()
        for read in cls.reads:
            kmers = read.generate_kmers(kmers_length)
            all_kmers.update(kmers)
        return list(all_kmers)
    generate_all_kmers = classmethod(generate_all_kmers)
    
    def generate_all_km1mers(cls):
        all_km1mers = set()
        for kmer in cls.all_kmers:
            all_km1mers.add(kmer[:-1]) #prefix
            all_km1mers.add(kmer[1:])  #suffix
        return list(all_km1mers)
    generate_all_km1mers = classmethod(generate_all_km1mers)
    
    
    



# class Graph:
#     def __init__(self):
#         self.edges = {} #Key : edge object, value : bool(True "is in graph", False "has been visited already")
#         self.vertices = []
#         self.cycles = []
        
#     def test_eulerian(self):
#         for vertex in self.vertices:
#             if len(vertex.edges_in) != len(vertex.edges_out):
#                 return False
#         return True
        
#     def choose_edge(self, list_edges):
#         for e in list_edges:
#             if self.edges[e]:
#                 return e
    
#     def find_cycle(self, edge):
#         cycle = [edge]
#         self.edges[edge] = False
#         le = edge.get_next_edges()
#         e = self.choose_edge(le)
#         while e != edge:
#             cycle.append(e)
#             self.edges[e] = False
#             le = e.get_next_edges()
#             e = self.choose_edge(le)
#         return cycle
    
#     def find_all_cycles(self):
#         while sum(self.edges.values()) > 0:
#             i = iter(self.edges.items())
#             edge, is_still_here = next(i)
#             while not is_still_here:
#                 edge, is_still_here = next(i)
#             cycle = self.find_cycle(edge)
#             self.cycles.append(cycle)
    
    
class Graph:
    def __init__(self, vertices, edges):
        self.vertices = vertices
        self.edges = edges
        self.euler_cycles = []
        self.euler_biggest = []
        
    def get_in_edges(self, vertex):
        in_edges=[]
        for edge in self.edges:
            if edge.vertex_to == vertex:
                in_edges.append(edge)
        return in_edges
        
    def get_out_edges(self, vertex):
        out_edges=[]
        for edge in self.edges:
            if edge.vertex_from == vertex:
                out_edges.append(edge)
        return out_edges
        
    def test_eulerian(self):
        for vertex in self.vertices:
            if len(self.get_in_edges(vertex)) != len(self.get_out_edges(vertex)):
                print('PROBLEM : This graph is not Eulerian.')
                return False
        print('SUCCESS : This graph is Eulerian !')
        return True
    
    def find_cycle(self):
        edge = self.edges[1]
        cycle = [edge]
        e = self.get_out_edges(edge.vertex_to)[0]
        while e != edge:
            cycle.append(e)
            self.edges.remove(e)
            e = self.get_out_edges(e.vertex_to)[0]
        self.edges.remove(e)
        return cycle
    
    def find_all_cycles(self):
        while len(self.edges) > 0:
            self.euler_cycles.append(self.find_cycle())
            
    def assemble_cycles(self, cycle, tab):
        # PROBLEM : contrairement a la version precedente, ici j'assemble des edges
        for e in cycle:
            self.euler_biggest.append(e)
            for c in tab:
                print(e in c)
                print(e)
                print(c)
                if e in c:
                    nextcycle = c[c.index(e)+1:] + c[:c.index(e)+1]
                    nexttab = tab[tab.index(c)+1:] + tab[:tab.index(c)]
                    self.assemble_cycles(nextcycle, nexttab)
                    tab.remove(c)
                    break
    
    
    
    
class Edge:
    edges = []
    
    def __init__(self, sequence):
        self.sequence = sequence
        self.vertex_from = None
        self.vertex_to = None
    
#     def get_next_edges(self):
#         return self.vertex_to.edges_out
            
        
class Vertex:
    vertices = []
    
    def __init__(self, sequence):
        self.sequence = sequence
#         self.edges_in = []
#         self.edges_out = []
    
    def get_vertex(cls, sequence):
        for vertex in cls.vertices:
            if vertex.sequence == sequence:
                return vertex
    get_vertex = classmethod(get_vertex)
    

In [56]:
# PROGRAMME PRINCIPAL
# Parameters
genome_length= 1000
read_length=7
reads_nb=20
kmers_length=3

# Genome generation
genome = Genome('ATGGCGTGCA')
print(genome.sequence)

# Reads generation
Read.reads = genome.sequencing(read_length=read_length, reads_nb=reads_nb)
# print(Read.reads)

# K-mers generation
Read.all_kmers = Read.generate_all_kmers(kmers_length=kmers_length)
print(Read.all_kmers)

# (K-1)-mers generation
Read.all_km1mers = Read.generate_all_km1mers()
print(Read.all_km1mers)

# Vertices generation
for km1mer in Read.all_km1mers:
    v = Vertex(km1mer)
    Vertex.vertices.append(v)
# print(Vertex.vertices)

# Edges generation
for kmer in Read.all_kmers:
    e = Edge(kmer)
    e.vertex_from = Vertex.get_vertex(e.sequence[:-1])     # prefix
    e.vertex_to = Vertex.get_vertex(e.sequence[1:])        # suffix
    Edge.edges.append(e)
print(len(Edge.edges))


# Graph generation
graph = Graph(Vertex.vertices, Edge.edges)
# print(graph.__dict__)

# Test Eulerien
print(graph.test_eulerian())

# Trouve tous les cycles
graph.find_all_cycles()
print(graph.euler_cycles)

# Assemble les cycles
graph.assemble_cycles(graph.euler_cycles[0], graph.euler_cycles[1:])


# Recréé la sequence


# Test genome de départ == genome assemblé




ATGGCGTGCA
['AAT', 'GGC', 'CGT', 'GCG', 'TGG', 'TGC', 'CAA', 'ATG', 'GTG', 'GCA']
['GC', 'TG', 'CA', 'GG', 'GT', 'AA', 'CG', 'AT']
10
SUCCESS : This graph is Eulerian !
True
[[<__main__.Edge object at 0x7f3a5f36b5f8>, <__main__.Edge object at 0x7f3a5f36b4e0>, <__main__.Edge object at 0x7f3a5f36b2b0>, <__main__.Edge object at 0x7f3a5f36bd30>, <__main__.Edge object at 0x7f3a5f36b320>], [<__main__.Edge object at 0x7f3a5f36b2e8>, <__main__.Edge object at 0x7f3a5f35fe48>, <__main__.Edge object at 0x7f3a5f36bba8>, <__main__.Edge object at 0x7f3a5f36b128>, <__main__.Edge object at 0x7f3a5f36b908>]]
False
False
False
False
False


---
## Tests

In [58]:
a = {2:True, 4:True, 5:False}